### Preprocesing of accident and incident data
- merge datasets
- add weather data
- filter data

In [1]:
import pandas as pd
import numpy as np
import datetime
import csv
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
paths = [
    "../data/1_raw/txt/accident_gravity_reporting_regulated_jul2020_present.txt",
    "../data/1_raw/txt/accident_hazardous_liquid_jan2010_present.txt",
    "../data/1_raw/txt/incident_gas_distribution_jan2010_present.txt",
    "../data/1_raw/txt/incident_gas_transmission_gathering_jan2010_present.txt",
    "../data/1_raw/txt/incident_liquefied_natural_gas_jan2011_present.txt",
    "../data/1_raw/txt/incident_type_r_reporting_regulated_gas_gathering_may2022_present.txt"
]

data_sources = [
    'gravity',
    'hazardous_liquid',
    'gas_distribution',
    'gas_transmission',
    'liquefied_natural_gas',
    'gas_gathering'
]

In [3]:
# concatenate tables to one df

df_temp1 = pd.read_csv(
    paths[0]
    , header=0
    , encoding='unicode_escape' 
    , sep='\t'
    #, index_col=['REPORT_NUMBER', 'SUPPLEMENTAL_NUMBER'] 
    , engine='python'
    )
df_temp1['data_source'] = data_sources[0]

df_temp2 = pd.read_csv(
    paths[1]
    , header=0
    , encoding='unicode_escape' 
    , sep='\t'
    #, index_col=['REPORT_NUMBER', 'SUPPLEMENTAL_NUMBER'] 
    , engine='python'
    )
df_temp2['data_source'] = data_sources[1]

df = pd.concat([df_temp1, df_temp2])

for path, data_source in zip(paths[2:], data_sources[2:]):
    df_temp = pd.read_csv(
        path
        , header=0
        , encoding='unicode_escape' 
        , sep='\t'
        #, index_col=['REPORT_NUMBER', 'SUPPLEMENTAL_NUMBER'] 
        , engine='python'
        )
    df_temp['data_source'] = data_source
    # convert mft to bbls
    df_temp['INTENTIONAL_RELEASE_BBLS'] =df_temp['INTENTIONAL_RELEASE'] * 178.10760668
    df_temp['UNINTENTIONAL_RELEASE_BBLS'] =df_temp['UNINTENTIONAL_RELEASE'] * 178.10760668  
    # drop columns
    df_temp.drop(labels=['INTENTIONAL_RELEASE', 'UNINTENTIONAL_RELEASE'], axis=1, inplace=True)
    df = pd.concat([df, df_temp])

print(df.shape)

(8874, 869)


In [4]:
# create columns 'case_date', 'YYYY-mm', 'YYYY-mm_prev'
df2 = df.copy()
df2['LOCAL_DATETIME'] = pd.to_datetime(df2['LOCAL_DATETIME'])
df2['case_date'] = df2['LOCAL_DATETIME'].dt.date
df2['YYYY-mm'] = pd.to_datetime(df2['case_date']).dt.strftime('%Y-%m')
df2['YYYY-mm_prev'] = (pd.to_datetime(df2['YYYY-mm']) - pd.DateOffset(months=1)).dt.strftime('%Y-%m')

df = df2.copy()

In [5]:
# drop unnamed columns
columns_to_drop = df.columns[df.columns.str.contains('Unnamed')]
df = df.loc[:, ~df.columns.isin(columns_to_drop)]

# rename some columns
df.rename(columns={'LOCATION_LATITUDE': 'case_lat', 'LOCATION_LONGITUDE': 'case_lon'}, inplace=True)

In [6]:
# drop dates outside of our range
# and records with no locations
df = df[
    (df['case_date'] >= datetime.date(2010, 1, 1)) 
    & (df['case_date'] <= datetime.date(2022, 12, 31))
    & (df['case_lat'].notnull())
    & (df['case_lon'].notnull())
    ]

In [7]:
# drop extreme locations
df = df[(df['case_lon'] <= -70) & (df['case_lon'] >= -152)]

In [8]:
# reset_index
df.reset_index(inplace=True, drop=True)

In [9]:
"""
# export to qgis for further processing
cols_qgis = [
             #'case_idx',
             'case_date',
             'case_lat',
             'case_lon',
             'COMMODITY_RELEASED_TYPE',
]

df_qgis = df[cols_qgis].copy()
# rename some columns
df_qgis.rename(columns={'COMMODITY_RELEASED_TYPE': 'rlsd_type'}, inplace=True)

df_qgis.to_csv('../data/1_raw/meteo/df_qgis.csv', index=True, index_label='case_idx')
#df_qgis.to_csv('../data/1_raw/meteo/df_qgis.csv', index=False)

"""
"""
PROCESSING IN QGIS:
    - assign nearest station to each observation
    - assign climate zone to each observation
"""

'\nPROCESSING IN QGIS:\n    - assign nearest station to each observation\n    - assign climate zone to each observation\n'

In [10]:
# import data on zones and weather
df_qgis_stations_zones = pd.read_csv('../data/1_raw/meteo/df_qgis_stations_zones.csv', header=0)

df_qgis_stations_zones.drop(['zones1_BA_Climate', 'zones2_BA_Climate', 'zones3_BA_Climate'], axis=1, inplace=True)

df_weather = pd.read_csv('../data/1_raw/meteo/weather.csv', header=0)

In [11]:
df_qgis_stations_zones.head()

,case_idx,case_date,case_lat,case_lon,rlsd_type,station_id,station_la,station_lo,station_el,zone
0,5427,2019/12/06,26.245950,-80.253315,NATURAL GAS,GHCND:USW00012885,26.19905,-80.17764,3.5,Hot-Humid
1,5965,2015/02/24,26.351202,-80.115319,NATURAL GAS,GHCND:USW00092805,26.24636,-80.11105,4.6,Hot-Humid
2,6799,2020/09/24,26.618578,-80.173433,NATURAL GAS,GHCND:USW00012844,26.68510,-80.09918,3.5,Hot-Humid
3,7768,2012/12/13,26.619750,-80.173490,NATURAL GAS,GHCND:USW00012844,26.68510,-80.09918,3.5,Hot-Humid
4,5318,2020/12/18,26.547920,-80.043020,NATURAL GAS,GHCND:USW00012844,26.68510,-80.09918,3.5,Hot-Humid


In [12]:
df_weather.head()

,YYYY-mm,station_id,TAVG,TMAX,TMIN,station_lat,station_lon
0,2010-01,GHCND:AQW00061705,29.40,31.92,26.87,-14.33056,-170.71361
1,2010-01,GHCND:CA001018611,7.57,9.43,5.72,48.03330,-123.33330
2,2010-01,GHCND:CA001135126,-0.66,2.10,-3.42,49.00000,-118.76670
3,2010-01,GHCND:CA001206197,-6.27,-3.53,-9.00,59.45000,-136.36670
4,2010-01,GHCND:CA005020881,-14.23,-10.51,-17.95,49.00000,-97.23330


In [13]:
# add station and zone data to df
df = pd.merge(df, df_qgis_stations_zones[['case_idx', 'station_id', 'zone']], how='left', left_index=True, right_on='case_idx')

In [14]:
df.head()

,REPORT_RECEIVED_DATE,IYEAR,REPORT_NUMBER,SUPPLEMENTAL_NUMBER,REPORT_TYPE,OPERATOR_ID,NAME,OPERATOR_STREET_ADDRESS,OPERATOR_CITY_NAME,OPERATOR_STATE_ABBREVIATION,...,PWJF_INSULATION_DEGRAD_IND,EQ_LOW_TEMP_EMBRITTLEMENT_IND,EQ_INSULATION_DEGRADATION_IND,MOP_NOT_DETERMINED_IND,case_date,YYYY-mm,YYYY-mm_prev,case_idx,station_id,zone
2311,1/30/2023,2022,20230010,39033,SUPPLEMENTAL FINAL,11551,"DELEK LOGISTICS OPERATING, LLC.",12700 PARK CENTRAL DRIVE SUITE 700,DALLAS,TX,...,NaN,NaN,NaN,NaN,2022-12-31,2022-12,2022-11,0,GHCND:USC00034185,Hot-Humid
2941,1/24/2023,2022,20230006,37618,ORIGINAL FINAL,31888,CENTURION PIPELINE L.P.,1300 MAIN STREET,HOUSTON,TX,...,NaN,NaN,NaN,NaN,2022-12-30,2022-12,2022-11,1,GHCND:USR0000TMID,Hot-Dry
2942,1/25/2023,2022,20230008,37797,SUPPLEMENTAL FINAL,15851,"DELEK MARKETING AND SUPPLY, LP",12700 PARK CENTRAL DRIVE SUITE 700,DALLAS,TX,...,NaN,NaN,NaN,NaN,2022-12-26,2022-12,2022-11,2,GHCND:USR0000TMID,Hot-Dry
4727,1/17/2023,2022,20230005,38456,SUPPLEMENTAL FINAL,22465,TARGA PIPELINE MID-CONTINENT WESTTEX LLC,"811 LOUISIANA STREET, SUITE 2100",HOUSTON,TX,...,NaN,NaN,NaN,NaN,2022-12-26,2022-12,2022-11,3,GHCND:USC00413411,Hot-Dry
474,1/17/2023,2022,20230004,37584,ORIGINAL FINAL,39649,"IRONWOOD MIDSTREAM ENERGY PARTNERS, LLC","15727 ANTHEM PARKWAY, SUITE 510",SAN ANTONIO,TX,...,NaN,NaN,NaN,NaN,2022-12-25,2022-12,2022-11,4,GHCND:USC00413873,Hot-Humid


In [15]:
# add weather data to df
df_ = pd.merge(df, df_weather[['YYYY-mm', 'station_id', 'TAVG']], how='left', on=['YYYY-mm', 'station_id'])
df2 = pd.merge(df_, df_weather[['YYYY-mm', 'station_id', 'TAVG']], how='left', left_on=['YYYY-mm_prev','station_id'], right_on=['YYYY-mm', 'station_id'])

df2.drop_duplicates(inplace=True)

In [16]:
df2.drop('YYYY-mm_y', axis=1, inplace=True)
df2.rename(columns={'Y-mm_x': 'YYYY-mm'}, inplace=True)

In [17]:
df2.head()

,REPORT_RECEIVED_DATE,IYEAR,REPORT_NUMBER,SUPPLEMENTAL_NUMBER,REPORT_TYPE,OPERATOR_ID,NAME,OPERATOR_STREET_ADDRESS,OPERATOR_CITY_NAME,OPERATOR_STATE_ABBREVIATION,...,EQ_INSULATION_DEGRADATION_IND,MOP_NOT_DETERMINED_IND,case_date,YYYY-mm_x,YYYY-mm_prev,case_idx,station_id,zone,TAVG_x,TAVG_y
0,1/30/2023,2022,20230010,39033,SUPPLEMENTAL FINAL,11551,"DELEK LOGISTICS OPERATING, LLC.",12700 PARK CENTRAL DRIVE SUITE 700,DALLAS,TX,...,NaN,NaN,2022-12-31,2022-12,2022-11,0,GHCND:USC00034185,Hot-Humid,8.51,NaN
1,1/24/2023,2022,20230006,37618,ORIGINAL FINAL,31888,CENTURION PIPELINE L.P.,1300 MAIN STREET,HOUSTON,TX,...,NaN,NaN,2022-12-30,2022-12,2022-11,1,GHCND:USR0000TMID,Hot-Dry,9.61,10.57
2,1/25/2023,2022,20230008,37797,SUPPLEMENTAL FINAL,15851,"DELEK MARKETING AND SUPPLY, LP",12700 PARK CENTRAL DRIVE SUITE 700,DALLAS,TX,...,NaN,NaN,2022-12-26,2022-12,2022-11,2,GHCND:USR0000TMID,Hot-Dry,9.61,10.57
3,1/17/2023,2022,20230005,38456,SUPPLEMENTAL FINAL,22465,TARGA PIPELINE MID-CONTINENT WESTTEX LLC,"811 LOUISIANA STREET, SUITE 2100",HOUSTON,TX,...,NaN,NaN,2022-12-26,2022-12,2022-11,3,GHCND:USC00413411,Hot-Dry,9.84,10.51
4,1/17/2023,2022,20230004,37584,ORIGINAL FINAL,39649,"IRONWOOD MIDSTREAM ENERGY PARTNERS, LLC","15727 ANTHEM PARKWAY, SUITE 510",SAN ANTONIO,TX,...,NaN,NaN,2022-12-25,2022-12,2022-11,4,GHCND:USC00413873,Hot-Humid,14.65,16.67


In [18]:
mapping = {'YES': 1, 'NO': 0}

cols_binary = [
    'IGNITE_IND',
    'EXPLODE_IND',
    'FEDERAL',
    'CROSSING',
    'COULD_BE_HCA',
    'SCADA_IN_PLACE_IND',
    'EMPLOYEE_DRUG_TEST_IND',
    'CONTRACTOR_DRUG_TEST_IND',]

In [19]:
df2[cols_binary] = df2[cols_binary].map(lambda x: mapping.get(x,x))

In [20]:
df2['case_date'] = pd.to_datetime(df2['case_date'])

In [21]:
# calculate installation age in days
df2['INSTALLATION_YEAR'].replace('UNKNOWN', np.nan, inplace=True)
df2['INSTALLATION_YEAR'] = pd.to_datetime(df2['INSTALLATION_YEAR'])

df2['inst_age_in_days'] = (df2['case_date'] - df2['INSTALLATION_YEAR']).dt.days

In [22]:
df2.loc[:, ['TAVG_x', 'TAVG_y']] = df2.loc[:, ['TAVG_x', 'TAVG_y']].fillna(0)
df2['TAVG'] = (df2['TAVG_x'] + df2['TAVG_y']) / 2
df2.drop(labels=['TAVG_x','TAVG_y'], axis=1, inplace=True)

In [23]:
# separate procedure to fill nans in TAVG using QGIS - by finding next nearest station

# import df of filled nans
df_filled_nans = pd.read_csv('../data/1_raw/meteo/df_filled_nans.csv', header=0)

In [24]:
# calculate avg temp from TAVG and TAVG_y

df_filled_nans.loc[:, ['TAVG', 'TAVG_y']] = df_filled_nans.loc[:, ['TAVG', 'TAVG_y']].fillna(0)

df_filled_nans['TAVG_x'] = (df_filled_nans['TAVG'] + df_filled_nans['TAVG_y']) / 2

In [25]:
# fill nans if df2
df2['TAVG'] = df2['TAVG'].fillna(df_filled_nans.set_index('case_idx')['TAVG_x'])

In [26]:
df2['TAVG'].isna().sum()

0

In [27]:
# check nans in ACCIDENT_PSIG and MOP_PSIG
print(df2['ACCIDENT_PSIG'].isna().sum()/df2['ACCIDENT_PSIG'].shape[0])
print(df2['MOP_PSIG'].isna().sum()/df2['MOP_PSIG'].shape[0])

0.1153184165232358
0.11790017211703958


In [28]:
df2['accident_pressure_as_%_mop_psig'] = df2['ACCIDENT_PSIG'] / df2['MOP_PSIG'] * 100
print(df2['accident_pressure_as_%_mop_psig'].isna().sum()/df2['accident_pressure_as_%_mop_psig'].shape[0])

0.14064420949102532


In [29]:
filtr = (df2['ACCIDENT_PSIG'] == 0) & (df2['MOP_PSIG'] == 0)
df2.loc[filtr, 'accident_pressure_as_%_mop_psig'] = 100

In [30]:
print(df2['accident_pressure_as_%_mop_psig'].isna().sum()/df2['accident_pressure_as_%_mop_psig'].shape[0])

0.11790017211703958


In [31]:
filtr = (df2['ACCIDENT_PSIG'].isna()) | (df2['MOP_PSIG'].isna())
df2.loc[filtr, 'accident_pressure_as_%_mop_psig'] = 100

print(df2['accident_pressure_as_%_mop_psig'].isna().sum()/df2['accident_pressure_as_%_mop_psig'].shape[0])

0.0


In [32]:
filtr = (df2['ACCIDENT_PSIG'] > 0) & (df2['MOP_PSIG'] == 0)
df2.loc[filtr, 'accident_pressure_as_%_mop_psig'] = 100

In [33]:
df2[['ACCIDENT_PSIG', 'MOP_PSIG', 'accident_pressure_as_%_mop_psig']].to_csv('processed_data_test.csv', index=True, index_label='case_idx')

In [34]:
nan_share_per_column = df2.isna().sum()/df2.shape[0]

cols_20prcnt_nan = nan_share_per_column[nan_share_per_column <= 0.2].index.tolist()
df_20prcnt_nan = df2.loc[:, cols_20prcnt_nan]

In [39]:
# manually selected columns
selected_cols = [
    'data_source',    
    'case_lat',
    'case_lon',
    'case_date',
    'INSTALLATION_YEAR',
    'COMMODITY_RELEASED_TYPE',
    'INTENTIONAL_RELEASE_BBLS',
    'UNINTENTIONAL_RELEASE_BBLS',    
    'FATAL',
    'INJURE',
    'ON_OFF_SHORE',
    'IGNITE_IND',
    'EXPLODE_IND',
    'NUM_PUB_EVACUATED',
    'FEDERAL',
    'LOCATION_TYPE',
    'CROSSING',
    'ITEM_INVOLVED',
    'MATERIAL_INVOLVED',
    'EST_COST_OPER_PAID',
    'EST_COST_GAS_RELEASED',
    'EST_COST_PROP_DAMAGE',
    'EST_COST_EMERGENCY',
    'EST_COST_ENVIRONMENTAL',
    'EST_COST_OTHER',
    'CAUSE',
    'CAUSE_DETAILS',
    'NARRATIVE',
    'SYSTEM_PART_INVOLVED',
    'INCIDENT_AREA_TYPE',
    'PIPE_FACILITY_TYPE',
    'inst_age_in_days',
    'RELEASE_TYPE',
    'COULD_BE_HCA',
    'ACCIDENT_PSIG',
    'MOP_PSIG',
    'accident_pressure_as_%_mop_psig',
    'PIPELINE_FUNCTION',
    'SCADA_IN_PLACE_IND',
    'INVESTIGATION_STATUS',
    'EMPLOYEE_DRUG_TEST_IND',
    'CONTRACTOR_DRUG_TEST_IND',
    'zone',
    'TAVG',
]

In [40]:
df2 = df2.loc[:, selected_cols]

In [41]:
print(df2.isna().sum()/df2.shape[0])

data_source                        0.000000
case_lat                           0.000000
case_lon                           0.000000
case_date                          0.000000
INSTALLATION_YEAR                  0.280920
COMMODITY_RELEASED_TYPE            0.011925
INTENTIONAL_RELEASE_BBLS           0.586550
UNINTENTIONAL_RELEASE_BBLS         0.009221
FATAL                              0.000000
INJURE                             0.000000
ON_OFF_SHORE                       0.166462
IGNITE_IND                         0.000000
EXPLODE_IND                        0.127858
NUM_PUB_EVACUATED                  0.068478
FEDERAL                            0.029875
LOCATION_TYPE                      0.029875
CROSSING                           0.120605
ITEM_INVOLVED                      0.166462
MATERIAL_INVOLVED                  0.090730
EST_COST_OPER_PAID                 0.002705
EST_COST_GAS_RELEASED              0.365626
EST_COST_PROP_DAMAGE               0.001598
EST_COST_EMERGENCY              

In [42]:
df2.to_csv('../data/2_merged/merged_data.csv', mode='w', index=True, index_label='case_idx')